In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Прочитаем данные

In [48]:
n = ['id', 'date','name','text','typr','rep','rtw','faw','stcount','foll','frien','listcount']
data_positive = pd.read_csv('positive.csv', sep=';', names=n)
data_negative = pd.read_csv('negative.csv', sep=';', names=n)

In [55]:
data = pd.concat([data_positive, data_negative])
# shuffling
df = data.sample(frac=1).reset_index(drop=True)
df

,id,date,name,text,typr,rep,rtw,faw,stcount,foll,frien,listcount
0,411073922784698368,1386842635,PFF___,"@apple_writer Смеяться очень полезно, так что ...",1,0,0,0,13,3,5,0
1,409761544663465985,1386529739,Du_Allein,"""@EpicBoojum: http://t.co/4vtaLlDEo2""\nАзаза, ...",1,0,0,0,342,7,16,0
2,415196483106115585,1387825530,Nik_nastena,"@Alexey_Tesnik сейчас огорчу тебя, твой подаро...",-1,0,0,0,6955,3364,342,18
3,410766861643833344,1386769426,julia061_,"Ехали с ним,и тут тот наш магазин с парковкой....",-1,0,0,0,815,44,48,0
4,410858451389411328,1386791262,AnyutaPuchkova,@vlad2lav7v @APolina1997 ахаха))\nХоть какой-т...,1,0,0,0,661,21,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,414058227706654720,1387554148,one_list1,"RT @lizaweasley19: давно не заходил сюда,и не ...",-1,0,1,0,3074,183,117,3
226830,411021558648676352,1386830150,AnyaLindemann,RT @RaMmKoTeIn98: @anyalindemann да уж:((( а н...,-1,0,1,0,3448,241,226,0
226831,412567059818897408,1387198626,torikrus211,"@Chemical_Nobody Ну не сильно,не научно популя...",-1,0,0,0,1448,10,54,0
226832,414016334683312128,1387544160,_s_t_r_o_n_g,блин только выздоровела и опять заболела :(,-1,0,0,0,4230,811,1290,4


Проведём предобработку текста (без подробного описания, просто некоторые преобразования текста + лемматизация)

In [57]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wandering/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

def remove_stopwords(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text


def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', text)
    text = re.sub('@[^\s]+','USER', text)
    text = text.lower().replace("ё", "е")
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +',' ', text)
    text = remove_stopwords(text)
    return text.strip()



In [59]:
lemmatized_text = df.text.apply(preprocess_text)

In [60]:
lemmatized_text

0         user смеяться очень полезно париться засмеятьс...
1                     user url азаз должный отписываться dd
2                      user огорчать твой подарок продавать
3         ехать наш магазин парковка выкидывать milka об...
4                      user user ахах плюс париться спасибо
                                ...                        
226829    rt user давно заходить сюда мочь сидеть долго ...
226830                                  rt user user летний
226831                user сильно научно популярно написать
226832                       блин выздоравливать заболевать
226833        хотеться проводить день любимый работа работа
Name: text, Length: 226834, dtype: object

In [61]:
df_enhanced = df
df_enhanced['text'] = lemmatized_text

In [62]:
df_enhanced

,id,date,name,text,typr,rep,rtw,faw,stcount,foll,frien,listcount
0,411073922784698368,1386842635,PFF___,user смеяться очень полезно париться засмеятьс...,1,0,0,0,13,3,5,0
1,409761544663465985,1386529739,Du_Allein,user url азаз должный отписываться dd,1,0,0,0,342,7,16,0
2,415196483106115585,1387825530,Nik_nastena,user огорчать твой подарок продавать,-1,0,0,0,6955,3364,342,18
3,410766861643833344,1386769426,julia061_,ехать наш магазин парковка выкидывать milka об...,-1,0,0,0,815,44,48,0
4,410858451389411328,1386791262,AnyutaPuchkova,user user ахах плюс париться спасибо,1,0,0,0,661,21,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,414058227706654720,1387554148,one_list1,rt user давно заходить сюда мочь сидеть долго ...,-1,0,1,0,3074,183,117,3
226830,411021558648676352,1386830150,AnyaLindemann,rt user user летний,-1,0,1,0,3448,241,226,0
226831,412567059818897408,1387198626,torikrus211,user сильно научно популярно написать,-1,0,0,0,1448,10,54,0
226832,414016334683312128,1387544160,_s_t_r_o_n_g,блин выздоравливать заболевать,-1,0,0,0,4230,811,1290,4


Финальный fit-predict:

In [96]:
train, test = train_test_split(df_enhanced, test_size=0.33, random_state=42)

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_text = vectorizer.fit_transform(train['text'])
test_text = vectorizer.transform(test['text'])

In [116]:
from sklearn import metrics
lr = LogisticRegression(solver = 'liblinear', random_state = 42, max_iter=1000)
lr.fit(train_text, train['typr'])
y_pred = lr.predict(test_text)
print("Accuracy: ",round(metrics.accuracy_score(test['typr'],y_pred),3))
print("F1: ",round(metrics.f1_score(test['typr'], y_pred),3))

Accuracy:  0.729
F1:  0.74


Выгрузим всё, дабы использовать в отдельном скрипте

In [122]:
import pickle
import os

with open('lr.pickle', 'wb') as f:
    pickle.dump(lr, f)
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)